<a href="https://colab.research.google.com/github/bivekSapkota/Constraint_Programming-for-Nursing-Depart/blob/main/CSUP_nursing_basic_model_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Ortools

In [ ]:
from ortools.sat.python import cp_model
model= cp_model.CpModel()

In [ ]:
# Code written by Bivek Sapkota

"""
The code has been corrected now, There were about 4 main errors and some minor errors:

Main Errors:
1: As Professor Bedoya pointed out, there were some issue with the code itself, in the constraint 2, "assign_junior[(group,week,day,block)])" was written assign_senior. Now corrected.
2: There was the issue with data. The demand for classes were more than available thus never reaching feasibility. Corrected by lowering the no. of groups for each seniority
3: There was the issue with modelling the problem itself, the 3rd constraint should not be associated with ad-- block availability. The block availabillity was removed.
4. Students with different seniority were being assigned to days not defined for them. Fixed by adding a new constraint(constraint2)

Minor Errors(not actually errors):
1. The code still does not incorporate the block_arrangement, which means for a particular day the group will be assigned to a random block and not a particular block. will correct later

 """

#Lets define decision variable and constants
no_of_weeks= 12 #no. of weeks in semester
no_of_days= 6 #no. of days per week
no_of_blocks= 4 # max no. of blocks per day, look at blocks_arrangement for per day arrangement
blocks_arrangement= [ # Maximum four blocks each day, this list can also be used to calculate the total blocks per day, just use sum(block_arrangement[day])
                     [0,1,0,0], #Monday
                     [1,1,1,1], #Tuesday
                     [1,1,1,1], #Wednesday
                     [0,0,1,0], #Thursday
                     [1,1,1,1], #Friday
                     [1,1,1,1]  #Saturday
                    ]

block_availability= [# M, T, W, Th,F, S   Defines how many blocks are available for each type of group for each day
                      [1, 0, 1, 0, 0, 1], #senior      {Mon, Wed, Sat}
                      [0, 1, 1, 1, 0, 1], #junior      {Tue, Wed, Thu, Sat}
                      [0, 1, 0, 1, 1, 1]  #accelerated {Tue, Thu, Fri, Sat}
                    ]

#defining total lab hours(actually the no. of blocks) required for each type of groups for the semester
Seniors_lab_hours= 4
juniors_lab_hours= 4          # its 4blocks not hours, where every block is 4 hours
accelerated_lab_hours= 7

#defining no. of groups for each seniority
no_of_senior_group= 9
no_of_junior_group= 11
no_of_accelerated_group= 7

days= ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

#Lets define Decision variables, Senior(Xgwdb), junior(Ygwdb), accelerated(Zgwdb) (separated to make the looping easier)
assign_senior= {}
for group in range(no_of_senior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        assign_senior[(group,week,day,block)]= model.NewBoolVar("Senior_group{}_week{}_day{}_block{}".format(group,week,day,block))

assign_junior= {}
for group in range(no_of_junior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        assign_junior[(group,week,day,block)]= model.NewBoolVar("Junior_group{}_week{}_day{}_block{}".format(group,week,day,block))

assign_accelerated= {}
for group in range(no_of_junior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        assign_accelerated[(group,week,day,block)]= model.NewBoolVar("Accelerated_group{}_week{}_day{}_block{}".format(group,week,day,block))

print("Number of variables =",(len(assign_senior)+len(assign_accelerated)+len(assign_junior)))



Number of variables = 8928


In [ ]:
#Lets define Constraints
#Constraint1 and 2: Each type of group(senior, junior and accelerated) need to attend defined no. of lab hours during the whole semester and should not attend the day not defined
for group in range(no_of_senior_group):
  seniortotalhours= []
  seniornoattendance=[]
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        seniortotalhours.append(assign_senior[(group,week,day,block)]) # index 0 in block_availability is for senior
        seniornoattendance.append(assign_senior[(group,week,day,block)]* (not block_availability[0][day]))
  model.Add(sum(seniortotalhours)== Seniors_lab_hours)
  model.Add(sum(seniornoattendance)==0)  # had to add this to not assign classes except the allowed days

for group in range(no_of_junior_group):
  juniortotalhours= []
  juniornoattendance=[]
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        juniortotalhours.append(assign_junior[(group,week,day,block)])
        juniornoattendance.append(assign_junior[(group,week,day,block)]*(not block_availability[1][day]))
  model.Add(sum(juniortotalhours)== juniors_lab_hours) # may also try greater than equal to later
  model.Add(sum(juniornoattendance)==0) # had to add this to not assign classes except the allowed days

for group in range(no_of_accelerated_group):
  acceleratedtotalhours= []
  acceleratednoattendance=[]
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      for block in range(no_of_blocks):
        acceleratedtotalhours.append(assign_accelerated[(group,week,day,block)])
        acceleratednoattendance.append(assign_accelerated[(group,week,day,block)]* (not block_availability[2][day]))
  model.Add(sum(acceleratedtotalhours)== accelerated_lab_hours)
  model.Add(sum(acceleratednoattendance)==0) # had to add this to not assign classes except the allowed days

#Constraint3: Every group can only be in 1 block per day
for group in range(no_of_senior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      seniorsinglesection=[]
      for block in range(no_of_blocks):
        seniorsinglesection.append(assign_senior[(group,week,day,block)])
      model.Add(sum(seniorsinglesection)<=1)

for group in range(no_of_junior_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      juniorsinglesection=[]
      for block in range(no_of_blocks):
        juniorsinglesection.append(assign_junior[(group,week,day,block)]) # first mistake was here
      model.Add(sum(juniorsinglesection)<=1)

for group in range(no_of_accelerated_group):
  for week in range(no_of_weeks):
    for day in range(no_of_days):
      acceleratedsinglesection=[]
      for block in range(no_of_blocks):
        acceleratedsinglesection.append(assign_accelerated[(group,week,day,block)]) #2nd mistake was here
      model.Add(sum(acceleratedsinglesection)<=1)

#constraint4: The lab hours available every day should not exceed
for week in range(no_of_weeks):
  for day in range(no_of_days):
    seniorlabHours= []
    juniorlabHours= []
    acceleratedlabHours= []
    for group in range(no_of_senior_group):
      for block in range(no_of_blocks):
        seniorlabHours.append(assign_senior[(group,week,day,block)])
    for group in range(no_of_junior_group):
      for block in range(no_of_blocks):
        juniorlabHours.append(assign_junior[(group,week,day,block)])
    for group in range(no_of_accelerated_group):
      for block in range(no_of_blocks):
        acceleratedlabHours.append(assign_accelerated[(group,week,day,block)])
    model.Add((sum(seniorlabHours)+sum(juniorlabHours)+sum(acceleratedlabHours))<=sum(blocks_arrangement[day]))


In [ ]:
#these 3 lines of code are the object for Solver unlike the object we created earlier for the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0         #for now I do not know what it does
solver.parameters.enumerate_all_solutions = True  # Enumerate all solutions. no idea what happens if I change the value to false

# now lets create the callback function
class callback(cp_model.CpSolverSolutionCallback):
  def __init__(self, assign_senior, assign_junior, assign_accelerated, no_of_days, no_of_blocks, no_of_weeks, no_of_senior_group, no_of_junior_group, no_of_accelerated_group, solution_limit, days):
    cp_model.CpSolverSolutionCallback.__init__(self)
    self._assign_senior= assign_senior
    self._assign_junior= assign_junior
    self._assign_accelerated= assign_accelerated
    self._no_of_days= no_of_days
    self._no_of_blocks= no_of_blocks
    self._no_of_weeks= no_of_weeks
    self._no_of_senior_group= no_of_senior_group
    self._no_of_junior_group= no_of_junior_group
    self._no_of_accelerated_group= no_of_accelerated_group
    self._solution_limit= solution_limit
    self._days= days
    self._solution_count= 0

  def on_solution_callback(self): # no idea if the name of this function can be changed, will try later
    self._solution_count+=1
    print("--------------------------------------------------------------------------------------------------------------------------------")
    print("Solution no.: {}".format(self._solution_count))
    print("--------------------------------------------------------------------------------------------------------------------------------")
    for week in range(self._no_of_weeks):
      print("Week{}: ".format(week+1))
      for day in range(self._no_of_days):
        print("\t",end="") # indentation
        print("\033[1m {} \033[0m".format(self._days[day]), end= ": ")  #Texts between \033[1m  and \033[0m  makes the texts bold
        for block in range(self._no_of_blocks):
          for group in range(self._no_of_senior_group):
            if self.Value(self._assign_senior[(group, week, day, block)]):
              print("Senior-Group{}".format(group+1), end= " |")
          for group in range(self._no_of_junior_group):
            if self.Value(self._assign_junior[(group, week, day, block)]):
              print("Junior-Group{}".format(group+1), end= " |")
          for group in range(self._no_of_accelerated_group):
            if self.Value(self._assign_accelerated[(group, week, day, block)]):
              print("Accelerated-Group{}".format(group+1), end= " |")
        print("\n")
    if self._solution_count>= self._solution_limit:
        self.StopSearch()
#-------------------------------------------------------Invoking the Solver------------------------------------------------------

#time to call the class, and solve
solution_limit= 1
mycallback= callback(assign_senior, assign_junior, assign_accelerated, no_of_days, no_of_blocks, no_of_weeks, no_of_senior_group, no_of_junior_group, no_of_accelerated_group, solution_limit, days)
solver.Solve(model, mycallback)



--------------------------------------------------------------------------------------------------------------------------------
Solution no.: 1
--------------------------------------------------------------------------------------------------------------------------------
Week1: 
	 Monday : 

	 Tuesday : Accelerated-Group5 |Accelerated-Group6 |Accelerated-Group7 |Accelerated-Group4 |

	 Wednesday : Senior-Group1 |

	 Thursday : Accelerated-Group7 |

	 Friday : Accelerated-Group5 |Accelerated-Group6 |Accelerated-Group7 |Accelerated-Group4 |

	 Saturday : Accelerated-Group4 |Accelerated-Group5 |Accelerated-Group6 |Accelerated-Group7 |

Week2: 
	 Monday : 

	 Tuesday : Accelerated-Group6 |Accelerated-Group7 |Accelerated-Group4 |

	 Wednesday : 

	 Thursday : Accelerated-Group4 |

	 Friday : Accelerated-Group4 |

	 Saturday : Accelerated-Group4 |Senior-Group1 |

Week3: 
	 Monday : 

	 Tuesday : 

	 Wednesday : Senior-Group8 |

	 Thursday : 

	 Friday : 

	 Saturday : Senior-Group8 |

Week

2